In [42]:
#ephemeris sim
using LinearAlgebra
#make sure to cite this package
using SPICE
using Downloads: download 
using DifferentialEquations
using Plots

In [43]:
using DelimitedFiles
xtraj  = readdlm("xtraj.txt", '\t', Float64, '\n')

#in days
time_steps = readdlm("time.txt", '\t', Float64, '\n')
#period 

T_periodic_n = 3.414975410587678

3.414975410587678

In [44]:
# Load leap seconds kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/naif0012.tls")

# laptop 
# furnsh("/home/fausto/naif0012.tls")

In [45]:
# Convert the calendar date to ephemeris seconds past J2000

#was 2018
#et = utc2et("2018-02-06T20:45:00")

et = 0

0

In [46]:
# Load a planetary ephemeris kernel
furnsh("/home/faustovega/Desktop/astrodynamics_nasa_work/de440.bsp")

#laptop
# furnsh("/home/fausto/de440.bsp")

In [47]:
#gravitational parameters for the bodies (km3/s2)

#moon 
μ_m = 4902.80058214776

#earth
μ_e = 398600.432896939

#sun
μ_s = 132712440017.987

1.32712440017987e11

In [48]:
#units used to normalize
#also in km and seconds

#distance
l_star = 385692.50

#time
t_star = 377084.152667038

#in km
lunar_radius = 1738.2 

1738.2

In [36]:
μ_earth_moon = 1.215058560962404E-2

0.01215058560962404

In [37]:
pose_m1 = [-μ_earth_moon*l_star , 0, 0]

3-element Vector{Float64}:
 -4686.389740239921
     0.0
     0.0

In [38]:
function hat(x)

    ω_hat = [0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0]

    return ω_hat
    
end

hat (generic function with 1 method)

In [39]:
#get the ECI state given a state in the rotating coordinate frame 
#all in km and s
function cr3bp_to_ephem(x_r, time)

    #state in the ECI ra
    x_eci = zeros(6)

    #time is already wrt et
    
    #state of the moon (position and velocity) relative to Earth (in km and km/s)
    moon_state = spkezr("moon",time,"J2000","NONE","earth")[1]

    #position of the moon relative to Earth
    r_moon = moon_state[1:3]

    #velocity of the moon relative to Earth
    v_moon = moon_state[4:6]

    #x component of the rotating frame expressed in inertial coordinates
    x̃ = r_moon/norm(r_moon)
    #z component of the rotating frame expressed in inertial coordinates
    z̃ = cross(r_moon, v_moon)/norm(cross(r_moon, v_moon))
    #x component of the rotating frame expressed in inertial coordinates
    ỹ = cross(z̃, x̃)

    #instantanous rotation matrix from rotating frame to inertial frame (centered at Earth)
    eci_R_r = [x̃ ỹ z̃]

    #cr3bp assumes that the rotating frame is rotating by the mean motion 

    θ_dot_mm = 2*pi/2.361e6
    #instantanous angular velocity
    #θ_dot = norm(cross(r_moon, v_moon))/(norm(r_moon)^2)
    #θ_dot_2 = (cross(r_moon, v_moon)/(norm(r_moon)^2))[3]

    #println("theta dot 2: ", θ_dot_2)

    ω = [0, 0, θ_dot_mm]

    ω_hat= hat(ω)
    
    #position transformation
    x_eci[1:3] = eci_R_r*(x_r[1:3] - pose_m1)

    x_eci[4:6] = eci_R_r*x_r[4:6] + eci_R_r*ω_hat*(x_r[1:3] - pose_m1)

    return x_eci
end

cr3bp_to_ephem (generic function with 1 method)

In [40]:
test = [431249.946141646,
0.0,
2286.769716989665,
0.0,
0.1811228748758214,
0.0]

6-element Vector{Float64}:
 431249.946141646
      0.0
   2286.769716989665
      0.0
      0.1811228748758214
      0.0

In [41]:
#this matches the pavlack paper
cr3bp_to_ephem(test, 0)

6-element Vector{Float64}:
 -315699.8428590055
 -289709.0899557205
  -80299.25699339015
       0.918770690350011
      -0.8884391404312567
      -0.4068140719442202

In [ ]:
#something is wrong with the integration...